In [30]:
from pathlib import Path
import sys
PROJECT_FOLDER = 'C:/Users/ryosh/enrich_me/project'
ORIGINAL_MODULES = PROJECT_FOLDER + '/modules'
sys.path.append(ORIGINAL_MODULES)

In [31]:
import MLDataset
import paths
import evaluate_model
from datetime import datetime

lasso_dataset = MLDataset.MLDataset(f'{paths.ML_DATASETS_FOLDER}/New48sectors')
lgbm_dataset = MLDataset.MLDataset(f'{paths.ML_DATASETS_FOLDER}/LGBM_after_New48sectors')
en_dataset = MLDataset.MLDataset(f'{paths.ML_DATASETS_FOLDER}/LGBM_New48sectors_Ensembled')
lgbm2_dataset = MLDataset.MLDataset(f'{paths.ML_DATASETS_FOLDER}/LGBM_test')

インスタンス情報を読み込みました。
インスタンス情報を読み込みました。
インスタンス情報を読み込みました。
インスタンス情報を読み込みました。


In [32]:
import machine_learning
en_dataset.pred_result_df['Pred'] = machine_learning.ensemble_by_rank(ml_datasets = [lasso_dataset, lgbm_dataset], 
                                                            ensemble_rates = [6.1, 1.9])

In [33]:
ml_dataset = en_dataset
model_obj = evaluate_model.LongShortModel(model_name = 'DJ_residuals', pred_result_df = ml_dataset.pred_result_df, 
                                          raw_target_df = ml_dataset.raw_target_df,
                                          start_date = datetime(2022, 1, 1), end_date = datetime.today(), 
                                          bin_num = 5, top_slope=1)
model_obj.display_result()

c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


DJ_residualsの読み込みが完了しました。
全期間
母集団の推定（99.7%）
平均リターン: 0.1441% (worst: 0.0841%, best: 0.2041%)
標準偏差: 0.00534 (worst:0.005796, best:0.004946)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.269816 (worst:0.145032, best:0.41272)
最大ドローダウン: 4.4533% (worst:8.9922%, best:2.6962%)
最大ドローダウン日数: 124日 (worst:428日, best:53日)
----------------------------------------------------------------------------------
直近1年
母集団の推定（99.7%）
平均リターン: 0.1472% (worst: 0.0444%, best: 0.25%)
標準偏差: 0.005446 (worst:0.006265, best:0.004803)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.270305 (worst:0.070859, best:0.520572)
最大ドローダウン: 4.5334% (worst:19.8933%, best:2.076%)
最大ドローダウン日数: 123日 (worst:1792日, best:33日)
----------------------------------------------------------------------------------
直近3ヶ月
母集団の推定（99.7%）
平均リターン: 0.1539% (worst: -0.0497%, best: 0.3576%)
標準偏差: 0.005232 (worst:0.007057, best:0.004114)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.294201 (worst:-0.070462, bes

Output()

In [34]:
en_dataset

In [35]:
import pandas as pd
pd.concat([lgbm_dataset.pred_result_df, en_dataset.pred_result_df], axis=1)

Target      Pred    Target  Pred
Date       Sector                                    
2014-01-06 ITインフラ  0.000233  0.000434  0.000233  44.0
           OA機器    0.008555  0.000507  0.008555  16.0
           その他工業   0.002350  0.000794  0.002350  34.0
           インフラ    0.001270 -0.000332  0.001270  17.0
           エンタメ    0.004108  0.000110  0.004108  39.0
...                     ...       ...       ...   ...
2024-11-13 電子部品         NaN -0.003154       NaN   4.0
           電気機器         NaN -0.000815       NaN  16.0
           電気機器2        NaN  0.000482       NaN  41.0
           非鉄金属         NaN -0.003552       NaN   1.0
           食品           NaN  0.001904       NaN  43.0

[127440 rows x 4 columns]

In [36]:
ml_dataset2 = MLDataset.MLDataset(f'{paths.ML_DATASETS_FOLDER}/LGBM_after_New48sectors.pkl.gz')

import pandas as pd
model = ml_dataset2.ml_models[0]
# モデルのトレーニング後に追加
importance_df = pd.DataFrame({
    'Feature': ml_dataset2.features_test_df.columns,
    'Importance': model.feature_importance(importance_type='gain')
}).sort_values(by='Importance', ascending=False)

pd.set_option('display.max_rows', None)
display(importance_df)
pd.reset_option('display.max_rows')

インスタンス情報を読み込みました。


TypeError: 'NoneType' object is not subscriptable